# Perfilamiento con time

In [2]:
import time

In [13]:
!pip install --quiet pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-1-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

You are using pip version 19.0.3, however version 21.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pprint
from scipy.optimize import linprog
from pytest import approx

from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, constrain, add_obj, obj, maxz, minz

## Objetivo

In [16]:
c_max_obj_1 = [-3, -5]
A_max_obj_1 = [[1, 0], [0, 2], [3, 2]]
b_max_obj_1 = [4, 12, 18]

In [17]:
max_obj_1 = -1*linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).fun

In [18]:
coeff_obj_1 = linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).x

## Aproximación

In [19]:
n_var_approx_1 = 2
n_cons_approx_1 = 3

In [20]:
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
end_time = time.time()
secs = end_time-start_time
print("create_matrix tomó",secs,"segundos" )

create_matrix tomó 0.0002834796905517578 segundos


In [21]:
start_time = time.time()
constrain(matrix_max_approx_1,'1,0,L,4')
end_time = time.time()
secs = end_time-start_time
print("constrain tomó",secs,"segundos" )

constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')

constrain tomó 0.0005183219909667969 segundos


In [22]:
start_time = time.time()
obj(matrix_max_approx_1,'3,5,0')
end_time = time.time()
secs = end_time-start_time
print("obj tomó",secs,"segundos" )

obj tomó 0.0004169940948486328 segundos


In [28]:
start_time = time.time()
problem_approx_1 = maxz(matrix_max_approx_1)
end_time = time.time()
secs = end_time-start_time
print("maxz tomó",secs,"segundos" )

maxz tomó 0.001978158950805664 segundos


In [30]:
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))

In [34]:
# Todo junto
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
constrain(matrix_max_approx_1,'1,0,L,4')
constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')
obj(matrix_max_approx_1,'3,5,0')
problem_approx_1 = maxz(matrix_max_approx_1)
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))
end_time = time.time()
secs = end_time-start_time
print("Todo el proceso tomó",secs,"segundos" )

Todo el proceso tomó 0.003371000289916992 segundos


## Comprobación

In [31]:
assert max_obj_1 == approx(max_approx_1), "El valor aproximado es incorrecto"
assert coeff_obj_1 == approx(coeff_approx_1), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_1)
print("El valor aproximado obtenido con mex es: ", max_approx_1)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_1)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1)

El valor objetivo obtenido con scipy es:  35.999999978433735
El valor aproximado obtenido con mex es:  36.0
Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


# Con comandos magic

Sólo probaremos con maxz que es la función que más se tardó en el proceso anterior

In [36]:
%time maxz(matrix_max_approx_1)

CPU times: user 653 µs, sys: 222 µs, total: 875 µs
Wall time: 888 µs


{'x1': 2.0, 'x2': 6.0, 'max': 36.0}

In [35]:
%timeit -n 5 -r 10 maxz(matrix_max_approx_1)

181 µs ± 54.3 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


# Con cProfile

In [1]:
import cProfile

In [37]:
cprof = cProfile.Profile()
cprof.enable()
problem_approx_1 = maxz(matrix_max_approx_1)
cprof.disable()
cprof.print_stats(sort='cumtime')

         151 function calls in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:3386(run_code)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <ipython-input-37-4199b605bc38>:3(<module>)
        1    0.000    0.000    0.000    0.000 problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:174(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:92(find_negative_row)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        7    0.000    0.000    0.000    0.000 simplex_networks.py:47(pivots_row)
       16    0.000    0.000    0.000    

## con comando magic

In [38]:
 %prun -s cumulative problem_approx_1 = maxz(matrix_max_approx_1)

In [41]:
cprof.dump_stats("maxz_stats")

In [42]:
import pstats

In [43]:
p_max_stats = pstats.Stats("maxz_stats")
print(p_max_stats.sort_stats("cumulative").print_stats(10))

Wed May  5 18:15:34 2021    maxz_stats

         151 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 39 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3386(run_code)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <ipython-input-37-4199b605bc38>:3(<module>)
        1    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 /home/user/.pyenv/versi

In [45]:
print(p_max_stats.sort_stats("cumulative").print_stats("simplex|lambda|math"))

Wed May  5 18:15:34 2021    maxz_stats

         151 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 39 to 8 due to restriction <'simplex|lambda|math'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py:178(maxz)
        4    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/simplex_networks.py:174(pivot)
        4    0.000    0.000    0.000    0.000 /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/simplex_networks.py:92(find_negative_row)
        7    0.000    0.000    0.000    0.000

In [46]:
print(p_max_stats.strip_dirs().sort_stats("cumulative").print_stats("lambda|simplex|math"))

Wed May  5 18:15:34 2021    maxz_stats

         151 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 39 to 7 due to restriction <'lambda|simplex|math'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:146(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:174(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:92(find_negative_row)
        7    0.000    0.000    0.000    0.000 simplex_networks.py:47(pivots_row)
        1    0.000    0.000    0.000    0.000 simplex_networks.py:26(pivots_col)
        6    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        2    0.000    0.000    0.000    0.000 interactiveshell.py:117(<lambda>)




In [48]:
# El número de veces que se llamaron a funciones primitivas o built in: 
print(p_max_stats.prim_calls)

151
